In [1]:
import pandas as pd
import numpy as np
file=pd.read_csv('C:\\Users\\zhou\\Desktop\\train.csv')
file1=pd.read_csv('C:\\Users\\zhou\\Desktop\\test.csv')
file2=pd.read_csv('C:\\Users\\zhou\\Desktop\\gender_submission.csv')

In [2]:
data_train=file.loc[0:,['Pclass','Sex','Age','SibSp','Parch','Fare']]
data_train_label=file.loc[0:,'Survived']
data_test=file1.loc[0:,['Pclass','Sex','Age','SibSp','Parch','Fare']]
data_test_label=file2.iloc[0:,0:]

In [3]:
count=0
for i in data_train.loc[0:,'Sex']:
    count+=1
    if i=='female':
        data_train.loc[0:,'Sex'][count-1]=0
    if i=='male':
        data_train.loc[0:,'Sex'][count-1]=1
count=0
for i in data_test.loc[0:,'Sex']:
    count+=1
    if i=='female':
        data_test.loc[0:,'Sex'][count-1]=0
    if i=='male':
        data_test.loc[0:,'Sex'][count-1]=1

In [4]:
data_others=np.array(data_train)[0:,[0,1,3,4,5]]
data_age=np.array(data_train)[0:,[2]]
data_test_others=np.array(data_test)[0:,[0,1,3,4,5]]
data_test_age=np.array(data_test)[0:,[2]]

In [5]:
nan_index_list=[]
for i in range(891):
    if pd.isnull(data_age[i][0]):
        nan_index_list.append(i)  
nan_test_index_list=[]
for i in range(418):
    if pd.isnull(data_test_age[i][0]):
        nan_test_index_list.append(i)  

In [6]:
#归一化处理
data_others=(data_others-data_others.min(axis=0))/(data_others.max(axis=0)-data_others.min(axis=0))
data_test_others=(data_test_others-data_test_others.min(axis=0))/(data_test_others.max(axis=0)-data_test_others.min(axis=0))

C:\Users\zhou\anaconda3\lib\site-packages\numpy\core\_methods.py:34: RuntimeWarning: invalid value encountered in reduce
  return umr_minimum(a, axis, None, out, keepdims, initial, where)
C:\Users\zhou\anaconda3\lib\site-packages\numpy\core\_methods.py:30: RuntimeWarning: invalid value encountered in reduce
  return umr_maximum(a, axis, None, out, keepdims, initial, where)


In [7]:
#利用knn算法补充age的缺失值
def knn(nums,data_others,k,data_age):
    data_size=data_others.shape[0]
    diffmat=np.tile(data_others[nums],(data_size,1))-data_others
    diffmat_sqr=diffmat**2
    sqrdiff=diffmat_sqr.sum(axis=1)
    distance_set=sqrdiff**0.5
    distance_sort=np.argsort(distance_set)
    list_near_average=[]
    for i in range(k):
        list_near_average.append(distance_sort[i])
    average=[]
    for i in list_near_average:
        average.append(data_age[i][0])
    average_new=[]
    for i in average:
        if pd.notnull(i):
            average_new.append(i)
    return sum(average_new)/len(average_new)
for nums in nan_index_list:
    data_train.loc[0:,'Age'][nums]=knn(nums,data_others,10,data_age)
for nums1 in nan_test_index_list:
    data_test.loc[0:,'Age'][nums1]=knn(nums1,data_test_others,10,data_test_age)

In [8]:
data_train=np.array(data_train)
data_train_label=np.array(data_train_label)

In [9]:
data_test=np.array(data_test)
data_test_label=np.array(data_test_label)
data_test_label=data_test_label[0:,1]

In [10]:
data_test.shape

(418, 6)

In [11]:
list_one=[]
for i in range(891):
    list_one.append(1)
list_one1=[]
for i in range(418):
    list_one1.append(1)

In [12]:
one=np.array(list_one)
one=one.reshape(891,1)

In [13]:
one1=np.array(list_one1).reshape(418,1)

In [14]:
data_train=np.hstack((one,data_train))
data_test=np.hstack((one1,data_test))

In [15]:
#构建神经网络

In [16]:
import torch
from torch.autograd import Variable
data_train=data_train.astype(float) 
data_train_label=data_train_label.astype(float)
data_test=data_test.astype(float)
data_test_label=data_test_label.astype(float)
data_train=torch.from_numpy(data_train) 
data_train_label=torch.from_numpy(data_train_label) 
data_test=torch.from_numpy(data_test)
data_test_label=torch.from_numpy(data_test_label)

In [17]:
def sigmoid(z):
    return (1)/(1+torch.exp(-z))

In [18]:
def neural_network(x,y):
    x=Variable(data_train,requires_grad=False)
    x=x.T
    y=Variable(data_train_label,requires_grad=False)
    w1=Variable(torch.randn(10,7,dtype=torch.float64),requires_grad=True)
    w2=Variable(torch.randn(10,10,dtype=torch.float64),requires_grad=True)
    w3=Variable(torch.randn(1,10,dtype=torch.float64),requires_grad=True)
    max_iter=10000
    learning_rate=0.0001
    for iter_nums in range(max_iter):
        z1=w1.mm(x)
        a1=sigmoid(z1)
        z2=w2.mm(a1)
        a2=sigmoid(z2)
        z3=w3.mm(a2)
        y_pred=sigmoid(z3)
        loss=((y-y_pred)**2).sum()
        loss.backward()
        w1.data-=learning_rate*w1.grad.data
        w2.data-=learning_rate*w2.grad.data
        w3.data-=learning_rate*w3.grad.data
        w1.grad.data.zero_()
        w2.grad.data.zero_()
        w3.grad.data.zero_()
    return w1,w2,w3

In [19]:
w1,w2,w3=neural_network(data_train,data_train_label)

In [20]:
w1

tensor([[ 2.8376, -0.1760, -0.6511, -0.6527,  0.7853, -0.0412, -0.7956],
        [-0.4500, -0.3247, -1.9401,  0.0075,  2.1725,  1.4217, -0.3903],
        [-0.5554,  0.7290, -1.2222, -0.0773,  0.0319, -0.7620, -0.9601],
        [ 1.9246,  1.2021,  0.9933,  2.3252, -0.6025,  1.6987,  2.1495],
        [ 1.5615,  0.2944,  1.7614, -1.0614,  1.4405, -1.1073, -0.5487],
        [-1.0669,  0.5164,  0.5425, -0.3051, -1.0405, -0.4197, -0.5199],
        [ 0.3973,  1.3750, -0.2641,  1.0951, -0.7589,  0.2172,  1.2493],
        [ 1.4318,  0.9906,  0.7802, -0.9137,  0.3620, -1.1183, -0.6668],
        [ 2.1452,  0.7693,  1.0279, -0.5467, -1.5577,  0.8724, -0.0336],
        [ 1.0930, -0.3437, -4.7325, -0.0190, -0.9913, -0.5850,  0.0487]],
       dtype=torch.float64, requires_grad=True)

In [21]:
w2

tensor([[-1.5967e-01,  7.9891e-01, -1.5646e-02,  1.0518e+00, -1.9057e+00,
          1.3303e+00,  2.0875e+00,  1.1068e+00,  4.8740e-01,  1.4069e+00],
        [ 1.5097e-01, -3.6754e-02, -5.0107e-01, -1.3032e+00,  6.1943e-01,
         -2.1910e+00,  4.8530e-01,  1.5482e-01, -2.7051e+00, -7.4515e-01],
        [-8.5869e-01,  2.3207e+00, -2.5316e-01,  9.1185e-02, -2.0845e+00,
         -2.0226e-01, -2.1971e+00,  1.0648e+00, -5.3478e-01, -9.6961e-01],
        [ 3.1358e-01,  6.6428e-01,  1.1868e+00, -5.0230e-01, -7.1651e-01,
          3.6501e-01, -1.0880e-01, -5.0977e-01,  7.7206e-01,  2.0057e+00],
        [-1.0877e+00,  2.4058e-01,  1.1182e-01,  9.9191e-01, -2.8706e-02,
         -8.3377e-01, -1.9792e-01, -4.5339e-01, -4.4635e-01, -1.7581e-01],
        [-1.5049e+00, -2.0252e+00,  8.2204e-01, -1.9359e-02, -1.2105e+00,
         -1.1866e+00, -4.2599e-01, -1.5567e+00, -1.4655e-01,  5.2497e-01],
        [-2.4975e+00,  5.8032e-01,  7.2832e-01,  6.8383e-01, -9.7769e-01,
         -1.2850e-01,  1.0574e+0

In [22]:
w3

tensor([[-1.0220, -1.7732, -0.4193,  2.4045,  0.6127,  0.1905,  0.0878,  0.4824,
         -3.2127,  0.8525]], dtype=torch.float64, requires_grad=True)

In [23]:
data_test.shape

torch.Size([418, 7])

In [24]:
def neural_network_pred(w1,w2,w3,data_test):
    data_test=data_test.T
    z1=w1.mm(data_test)
    a1=sigmoid(z1)
    z2=w2.mm(a1)
    a2=sigmoid(z2)
    z3=w3.mm(a2)
    y_pred=sigmoid(z3)
    return y_pred
    
    

In [25]:
pre=neural_network_pred(w1,w2,w3,data_test)

In [26]:
pre

tensor([[0.1387, 0.3250, 0.1383, 0.1393, 0.3191, 0.1779, 0.5737, 0.1382, 0.6230,
         0.1363, 0.1392, 0.1487, 0.8214, 0.1374, 0.7964, 0.6615, 0.1411, 0.1405,
         0.3831, 0.4995, 0.1573, 0.7117, 0.7826, 0.2121, 0.8283, 0.1364, 0.8160,
         0.1399, 0.1541, 0.1362, 0.1381, 0.1375, 0.2415, 0.3509, 0.1690, 0.1432,
         0.6099, 0.6217, 0.1397, 0.1800, 0.1373, 0.1504, 0.1383, 0.6882, 0.7804,
         0.1395, 0.1519, 0.1417, 0.8214, 0.3553, 0.1782, 0.1431, 0.2821, 0.8283,
         0.1425, 0.1375, 0.1387, 0.1395, 0.1371, 0.8283, 0.1475, 0.1418, 0.1442,
         0.6091, 0.8274, 0.7476, 0.6284, 0.1662, 0.1561, 0.8283, 0.6009, 0.1405,
         0.5810, 0.1563, 0.8283, 0.8277, 0.1391, 0.3501, 0.1419, 0.6009, 0.6669,
         0.8263, 0.1479, 0.1392, 0.1418, 0.1370, 0.5893, 0.6298, 0.5749, 0.7537,
         0.4375, 0.1396, 0.7071, 0.1391, 0.1559, 0.1395, 0.8052, 0.1391, 0.6188,
         0.1388, 0.7887, 0.1400, 0.1417, 0.1394, 0.3796, 0.1430, 0.1405, 0.1417,
         0.1403, 0.1449, 0.1

In [27]:
pre=pre.detach().numpy()

In [28]:
data_test_label=data_test_label.detach().numpy()
pre.reshape(418,)

array([0.13870066, 0.32504114, 0.13832915, 0.13932653, 0.31906636,
       0.17793414, 0.57370944, 0.13819675, 0.623033  , 0.13625661,
       0.13919901, 0.14872602, 0.821393  , 0.13740236, 0.79637946,
       0.66149843, 0.1410542 , 0.14051966, 0.38308555, 0.49948436,
       0.15731748, 0.71174667, 0.78262241, 0.21214912, 0.82830309,
       0.13641655, 0.81604834, 0.139926  , 0.15407567, 0.13624193,
       0.13813151, 0.13745019, 0.24149463, 0.35086812, 0.16896285,
       0.14319985, 0.60989961, 0.62169719, 0.13965902, 0.18001392,
       0.13734342, 0.15037167, 0.13830428, 0.6882409 , 0.78039262,
       0.13948204, 0.15190578, 0.14171504, 0.82137039, 0.35533312,
       0.1781944 , 0.14305361, 0.28213727, 0.82828644, 0.14252247,
       0.13753915, 0.1386698 , 0.13946624, 0.13706769, 0.82831248,
       0.14745556, 0.14176905, 0.14424783, 0.60909184, 0.82744452,
       0.74756232, 0.6283783 , 0.16621009, 0.15607526, 0.82829134,
       0.60086484, 0.14054065, 0.58097587, 0.15625056, 0.82830

In [29]:
for i in range(418):
    if pre[0][i]>=0.5:
        pre[0][i]=1
    if pre[0][i]<0.5:
        pre[0][i]=0

In [30]:
data_test_label=data_test_label.tolist()

In [31]:
pre=pre.tolist()

In [32]:
pre=pre[0]

In [33]:
count1=0
for i in range(len(pre)):
    if pre[i]==data_test_label[i]:
        count1+=1
print(count1/len(pre))

0.868421052631579


In [34]:
data_train=data_train.detach().numpy()
data_train_label=data_train_label.detach().numpy()
    

In [35]:
import random
from random import random

In [36]:
def auto(x,deriv=False):
    if deriv==True:
        return x*(1-x)
    return 1/(1+np.exp(-x))
def auto_grad(x,a1,a2,y_pred,y):
    error_y=y-y_pred
    delta_y=error_y*auto(y_pred,deriv=True)

    error_a2=delta_y.dot(w3.T)
    delta_a2=error_a2*auto(a2,deriv=True)

    error_a1=delta_a2.dot(w2.T)
    delta_a1=error_a1*auto(a1,deriv=True)
    return x.T.dot(delta_a1),a1.T.dot(delta_a2),a2.T.dot(delta_y)

In [37]:
def neural_network_2(x,y):
    m,n=x.shape
    np.random.seed(1)
    w_1=2*np.random.random((n,10))-1
    w_2=2*np.random.random((10,10))-1
    w_3=2*np.random.random((10,1))-1
    for i in range(1000):
        a1=auto(np.dot(x,w_1))
        a2=auto(np.dot(a1,w_2))
        y_pred=auto(np.dot(a2,w_3))
        error_y=y-y_pred
        delta_y=error_y*auto(y_pred,deriv=True)

        error_a2=delta_y.dot(w_3.T)
        delta_a2=error_a2*auto(a2,deriv=True)

        error_a1=delta_a2.dot(w_2.T)
        delta_a1=error_a1*auto(a1,deriv=True)
        
        
        
        w_1+=x.T.dot(delta_a1)
        w_2+=a1.T.dot(delta_a2)
        w_3+=a2.T.dot(delta_y)
    return w_1,w_2,w_3
        

In [ ]:
w1,w2,w3=neural_network_2(data_train,data_train_label)

In [ ]:
w1_change,w2_change,w3_change=auto_grad(x,a1,a2,y_pred,y)